In [ ]:
import os
import numpy as np
import pandas as pd
from datetime import datetime
os.chdir("../")
from datasets import load_dataset, Dataset, DatasetDict
from src.data import HF_Dataset, HF_Dataset_Params
from src.utils import get_split_limit

In [ ]:
dataset = load_dataset("monash_tsf", "tourism_monthly")
freq = "1H"
prediction_length = 48

In [ ]:
dataset["train"]

In [ ]:
params = HF_Dataset_Params(
    [datetime(2023, 4, 12, 12, 0, 0), datetime(2023, 4, 12, 12, 0, 0)], 
    [[i for i in range(30)], [-i for i in range(30)]],
    [[], []],
    [[], []],
    np.array([0.7, 0.2, 0.1]),
    "1H"
)
ds_obj = HF_Dataset(params)

In [ ]:
multi_variate_train_example, multi_variate_test_example = ds_obj.multi_variate_datasets()

In [ ]:
multi_variate_train_example

In [ ]:
multi_variate_test_example

In [ ]:
from transformers import InformerConfig, InformerForPrediction

config = InformerConfig(
    # in the multivariate setting, input_size is the number of variates in the time series per time step
    input_size=num_of_variates,
    # prediction length:
    prediction_length=prediction_length,
    # context length:
    context_length=prediction_length * 2,
    # lags value copied from 1 week before:
    lags_sequence=[1, 24 * 7],
    # we'll add 5 time features ("hour_of_day", ..., and "age"):
    num_time_features=len(time_features) + 1,
    # informer params:
    dropout=0.1,
    encoder_layers=6,
    decoder_layers=4,
    # project input from num_of_variates*len(lags_sequence)+num_time_features to:
    d_model=64,
)

model = InformerForPrediction(config)